# Advent of Code 2022

I liked [Peter Norvig's approach](https://github.com/norvig/pytudes/blob/main/ipynb/Advent-2020.ipynb) [last year](https://github.com/codemonkeyjim/adventofcode-2021/blob/main/aoc-2021.ipynb), so I'm going to use it again this year.

## Day 0: Imports and Utility Functions
Preparations prior to Day 1:

- Some imports.
- A way to read the day's data file and to print/check the output.
- Some utilities that are likely to be useful.


In [ ]:
from __future__ import annotations
from collections import Counter, defaultdict, namedtuple
from dataclasses import dataclass
from functools import reduce
from itertools import chain, permutations
from math import prod
import operator
from queue import PriorityQueue
from statistics import mean, median
from typing import Callable


In [ ]:
def data(day: int, parser=str, sep='\n', filetype="input") -> list:
    "Split the day's input file into sections separated by `sep`, and apply `parser` to each."
    sections = open(f'data/advent2022/{filetype}{day}.txt').read().rstrip().split(sep)
    return [parser(section) for section in sections]
     
def do(day, *answers) -> dict[int, int]:
    "E.g., do(3) returns {1: day3_1(in3), 2: day3_2(in3)}. Verifies `answers` if given."
    g = globals()
    got = []
    for part in (1, 2):
        fname = f'day{day}_{part}'
        if fname in g: 
            got.append(g[fname](g[f'in{day}']))
            if len(answers) >= part: 
                assert got[-1] == answers[part - 1], (
                    f'{fname}(in{day}) got {got[-1]}; expected {answers[part - 1]}')
    return got

def by_line(text: str) -> list[str]:
    "Split the text into a list of lines."
    return text.strip().splitlines()

def first(iterable, default=None) -> object:
    "Return first item in iterable, or default."
    return next(iter(iterable), default)

def rest(sequence) -> object: return sequence[1:]



## Day 1: Calorie Counting

1. Find the Elf carrying the most Calories. How many total Calories is that Elf carrying?
2. Find the top three Elves carrying the most Calories. How many Calories are those Elves carrying in total?

In [ ]:
in1 = data(1, parser=lambda lines: [int(line) for line in lines.split("\n")], sep="\n\n")

In [ ]:
def day1_1(packs: list(int)) -> int:
    return max([sum(pack) for pack in packs])

In [ ]:
def day1_2(packs: list(int)) -> int:
    return sum(sorted([sum(pack) for pack in packs])[-3:])

In [ ]:
do(1, 69626)